In [1]:
import pandas as pd
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = pd.read_csv('datasets/DarijaEnglish.csv')
dataset

,Darija,English
0,واخا عندي السخانة والتهاب الحلق، ولكن حاس بالل...,"""Apart from the fever and a sore throat, I fee..."
1,الاستناد لهاذ الحفرية كيعني أن الانشقاق وقع بك...,"""Based on this fossil, that means the split is..."
2,"وقال السيد كوستيلو: ""يلا ولات القضية تجارية، خ...","""If it becomes commercial, we should have it. ..."
3,"قال المتحدث باسم الحكومة: ""من المهم دابا سواءً...","""It is now important from both a public health..."
4,غادير لحال سلوك دالما. هي شفافة تماما بحال الما.,"""It's going to behave like water. It's transpa..."
...,...,...
6022,الشركة نتاع التيليفون لمحلية ديالك خاصها تعطيك...,Your local telephone service provider should b...
6023,يقدم الغار نفسه، اللي بقي على مر العصور، صورة ...,"he cave itself, which survived the times, give..."
6024,فأواخر العصور الوسطى، بدات أوروبّا كتطور فراسه...,towards the end of the Middle Ages western Eur...
6025,إيزمير هي ثالث أكبر مدينة فتركيا بعدد سكان تيو...,İzmir is the third largest city in Turkey with...


In [ ]:
%%capture

!pip install fairseq2
!pip install pydub sentencepiece
!pip install git+https://github.com/facebookresearch/seamless_communication.git

In [ ]:
# Initialize a Translator object with a multitask model, vocoder on the GPU.

from seamless_communication.inference import Translator

model_name = "seamlessM4T_v2_large"

translator = Translator(
    model_name,
    None,
    device=device,
    dtype=torch.float16,
)

100%|██████████| 8.45G/8.45G [01:36<00:00, 93.7MB/s]
100%|██████████| 360k/360k [00:00<00:00, 10.1MB/s]
100%|██████████| 4.93M/4.93M [00:00<00:00, 66.6MB/s]
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.


In [ ]:
def english_to_msa(english_sentence):
  text_output, _ = translator.predict(
      input=english_sentence,
      task_str="t2tt",
      tgt_lang='arb',
      src_lang="eng",
  )

  return str(text_output[0])

In [ ]:
from tqdm import tqdm
MSA_translations = []
for i in tqdm(range(dataset.shape[0])):
  MSA_translations.append(english_to_msa(dataset.loc[i].English))

100%|██████████| 6027/6027 [1:33:20<00:00,  1.08it/s]


In [ ]:
dataset['MSA'] = MSA_translations
dataset.to_csv('datasets/Augmented_dataset.csv', index=False)

In [3]:
augmented_dataset = pd.read_csv('datasets/Augmented_dataset.csv')
augmented_dataset.head(4)

,Darija,English,MSA
0,واخا عندي السخانة والتهاب الحلق، ولكن حاس بالل...,"""Apart from the fever and a sore throat, I fee...","""بصرف النظر عن الحمى وألم الحلق ، أشعر أنني في..."
1,الاستناد لهاذ الحفرية كيعني أن الانشقاق وقع بك...,"""Based on this fossil, that means the split is...","""بناءً على هذا الحفري ، هذا يعني أن الانقسام أ..."
2,"وقال السيد كوستيلو: ""يلا ولات القضية تجارية، خ...","""If it becomes commercial, we should have it. ...","""إذا أصبح تجاريًا ، يجب أن نحصل عليه ، وهذا يع..."
3,"قال المتحدث باسم الحكومة: ""من المهم دابا سواءً...","""It is now important from both a public health...","""من المهم الآن من وجهة نظر الصحة العامة والعدا..."


In [4]:
original_dataset = pd.read_csv('datasets/Original_dataset.csv')
original_dataset.head(4)

,idx,English,Darija,MSA,Checkbox,Darija corrected,Observations
0,0,"They're hiding something, I'm sure!","هوما مخبّيين شي حاجة, أنا متيقّن!",إنهم يخفون شيئًا، أنا متأكد!,True,NaN,NaN
1,1,It's obvious they're trying to keep their cool.,باينا هوما تايحاولو إبقاو مبرّدين.,من الواضح أنهم يحاولون الحفاظ على هدوئهم.,True,NaN,NaN
2,2,the hotels don't seem very comfortable.,لوطيلات مابايناش فيهوم موريحين بزّاف.,الفنادق لا تبدو مريحة جدًا.,True,NaN,NaN
3,3,he is probably about to be laid off by head of...,غاليبان غايجرّيو عليه من لخدما!,من المحتمل أن يتم تسريحه من قبل الإدارة العليا!,False,غاليبان غايجرّيو عليه من اللفوق,NaN


In [5]:
def get_darija(row):
  if not pd.isnull(row['Darija corrected']):
    return row['Darija corrected']
  else:
    return row['Darija']

In [6]:
original_dataset['Darija'] = original_dataset.apply(get_darija, axis=1)
original_dataset = original_dataset[['Darija', 'English', 'MSA']]
original_dataset.head(4)

,Darija,English,MSA
0,"هوما مخبّيين شي حاجة, أنا متيقّن!","They're hiding something, I'm sure!",إنهم يخفون شيئًا، أنا متأكد!
1,باينا هوما تايحاولو إبقاو مبرّدين.,It's obvious they're trying to keep their cool.,من الواضح أنهم يحاولون الحفاظ على هدوئهم.
2,لوطيلات مابايناش فيهوم موريحين بزّاف.,the hotels don't seem very comfortable.,الفنادق لا تبدو مريحة جدًا.
3,غاليبان غايجرّيو عليه من اللفوق,he is probably about to be laid off by head of...,من المحتمل أن يتم تسريحه من قبل الإدارة العليا!


In [7]:
dataset = pd.concat([original_dataset, augmented_dataset], axis=0)
dataset

,Darija,English,MSA
0,"هوما مخبّيين شي حاجة, أنا متيقّن!","They're hiding something, I'm sure!",إنهم يخفون شيئًا، أنا متأكد!
1,باينا هوما تايحاولو إبقاو مبرّدين.,It's obvious they're trying to keep their cool.,من الواضح أنهم يحاولون الحفاظ على هدوئهم.
2,لوطيلات مابايناش فيهوم موريحين بزّاف.,the hotels don't seem very comfortable.,الفنادق لا تبدو مريحة جدًا.
3,غاليبان غايجرّيو عليه من اللفوق,he is probably about to be laid off by head of...,من المحتمل أن يتم تسريحه من قبل الإدارة العليا!
4,طابعان راه مكتاءب!,Of course he's depressive!,بالطبع هو مكتئب!
...,...,...,...
6022,الشركة نتاع التيليفون لمحلية ديالك خاصها تعطيك...,Your local telephone service provider should b...,يجب أن يكون مزود خدمة الهاتف المحلي قادرًا على...
6023,يقدم الغار نفسه، اللي بقي على مر العصور، صورة ...,"he cave itself, which survived the times, give...",الكهف نفسه ، الذي نجا من العصور ، يعطي صورة حي...
6024,فأواخر العصور الوسطى، بدات أوروبّا كتطور فراسه...,towards the end of the Middle Ages western Eur...,نحو نهاية العصور الوسطى ، بدأت أوروبا الغربية ...
6025,إيزمير هي ثالث أكبر مدينة فتركيا بعدد سكان تيو...,İzmir is the third largest city in Turkey with...,إزمير هي ثالث أكبر مدينة في تركيا ، ويبلغ عدد ...


In [8]:
dataset.to_csv('datasets/training_dataset.csv')